# BCxP GenAI Hands-on Sessions
## Part 4: Generative UI

Generative UI bezeichnet eine Benutzerschnittstelle, die sich dynamisch und kontextabhängig durch generative KI-Modelle anpassen oder erzeugen lässt 

### Beispiel
Ein Nutzer fragt:
„Zeig mir meine Verkaufszahlen der letzten 3 Monate im Vergleich zu Q1 2024.“
→ Die generative UI erstellt automatisch ein Dashboard mit dem passenden Plot, Filter und Interpretation – ohne dass das Layout oder die Abfrage vorher explizit programmiert war.

Im Folgenden wollen wir dem Nutzer ermöglichen Plots zu prompten und damit eine einfache Möglichkeit bieten den bereitgestellten Datensatz von Pizzaverkaufszahlen zu analysieren.

**Aufgabe:** Optimiere den System-Prompt, um fehlerhaften Code im Output zu vermeiden. Welche zusätzlichen Regeln könntest du hinzufügen? 

In [ ]:
import pandas as pd

# Lade Pizzaverkäufe-Daten aus einer CSV-Datei aus GitHub
url = "https://raw.githubusercontent.com/santoshkr23/Pizzasales/main/pizza_sales_dataset.csv"
df = pd.read_csv(url)

# Zeige Informationen über den DataFrame an 
df.info()
# Zeige die ersten 5 Zeilen des DataFrames an
df.head()

In [ ]:
import io
from openai import OpenAI
import sys
sys.path.append("..")
from hack_helpers import OPENAI_API_KEY
openai_client = OpenAI(
    api_key=OPENAI_API_KEY
)

system_prompt = """
# General Task Description
You are a strict and accurate Python code generator. Your task is to write complete Python code that - based on the provided pandas DataFrame (df) that includes pizza sales data and a user-specified plot request - produces a corresponding plotly.graph_objects.Figure.
The code must:
	1.	Transform the provided df through filtering, grouping, aggregating, or other methods to prepare the data for visualization.
	2.	Generate a well-styled plotly.graph_objects.Figure that fully satisfies the user's description and uses the transformed data.

# Output Rules (must be followed exactly)
    - Output only a single code block with executable Python code.
    - The code block must start without indentation.
    - Never add '```python' or '```' to the output. Directly start with the code!
    - Do not include explanations, markdown, comments, or pseudocode.
    - Do not add imports — assume pandas and plotly.graph_objects are already imported.
    - Do not redefine the df variable — use it directly as described in the provided df.info() and df.head() output.
    - Assign the result to the variable fig.
    - Ensure correct syntax.
    - Ensure every opening bracket has a corresponding closing bracket.
    - Respect all stylistic instructions given in the user's request.
    - Try to run the code yourself before outputting it to verify that it works correctly.
    - Only use Column names from the df.info() output and the df.head() output as provided in 
    - Always use the latest version of plotly.
    - Always ensure that formatting is correct according to colum defs in df.info() output.

# Data
## df.info() output
    <class 'pandas.core.frame.DataFrame'>
    RangeIndex: 48620 entries, 0 to 48619
    Data columns (total 12 columns):
    #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
    0   pizza_id           48620 non-null  int64  
    1   order_id           48620 non-null  int64  
    2   pizza_name_id      48620 non-null  object 
    3   quantity           48620 non-null  int64  
    4   order_date         48620 non-null  object 
    5   order_time         48620 non-null  object 
    6   unit_price         48620 non-null  float64
    7   total_price        48620 non-null  float64
    8   pizza_size         48620 non-null  object 
    9   pizza_category     48620 non-null  object 
    10  pizza_ingredients  48620 non-null  object 
    11  pizza_name         48620 non-null  object 
    dtypes: float64(2), int64(3), object(7)
    memory usage: 4.5+ MB

## df.head() output
    pizza_id	order_id	pizza_name_id	quantity	order_date	order_time	unit_price	total_price	pizza_size	pizza_category	pizza_ingredients	pizza_name
    0	1	1	hawaiian_m	1	01-01-2015	11:38:36	13.25	13.25	M	Classic	Sliced Ham, Pineapple, Mozzarella Cheese	The Hawaiian Pizza
    1	2	2	classic_dlx_m	1	01-01-2015	11:57:40	16.00	16.00	M	Classic	Pepperoni, Mushrooms, Red Onions, Red Peppers,...	The Classic Deluxe Pizza
    2	3	2	five_cheese_l	1	01-01-2015	11:57:40	18.50	18.50	L	Veggie	Mozzarella Cheese, Provolone Cheese, Smoked Go...	The Five Cheese Pizza
    3	4	2	ital_supr_l	1	01-01-2015	11:57:40	20.75	20.75	L	Supreme	Calabrese Salami, Capocollo, Tomatoes, Red Oni...	The Italian Supreme Pizza
    4	5	2	mexicana_m	1	01-01-2015	11:57:40	16.00	16.00	M	Veggie	Tomatoes, Red Peppers, Jalapeno Peppers, Red O...	The Mexicana Pizza


# Few Shot Examples

## Example 1: 
    <input>
        Show a bar plot of the 5 worst sold pizzas based on pizza name. Display also the ingredients of those pizzas on hover. Display the total number very big in the middle of each bar.
    </input>
    <output>
        worst_sold = df.groupby('pizza_name', as_index=False)['quantity'].sum().nsmallest(5, 'quantity')
        fig = go.Figure(
        data=[
            go.Bar(
                x=worst_sold['pizza_name'],
                y=worst_sold['quantity'],
                text=worst_sold['quantity'],
                textposition='inside',
                textfont=dict(size=24),
                hovertext=worst_sold['pizza_name'].map(
                    df.drop_duplicates('pizza_name').set_index('pizza_name')['pizza_ingredients']
                ),
                hoverinfo='text'
                )
            ]
        ) 
    </output>

## Example 2:
    <input>
        Show orders per hour (order_time) throughout the day.
    </input>
    <output>
        df['order_time'] = pd.to_datetime(df['order_time'], format='%H:%M:%S').dt.hour
        orders_per_hour = df.groupby('order_time', as_index=False)['order_id'].count()
        fig = go.Figure(
            data=[
                go.Bar(
                    x=orders_per_hour['order_time'],
                    y=orders_per_hour['order_id'],
                    text=orders_per_hour['order_id'],
                    textposition='outside'
                )
            ]
        )
    </output>
"""


def generate_plot(df: pd.DataFrame, user_input: str) -> str: 
    """
    Generates a plotly figure based on the provided DataFrame and user input.
    
    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        user_input (str): The user input describing the desired plot.
    
    Returns:
        Response: the python code generated by the OpenAI API as str
    """

    # Capture df.info() output as a string
    buf = io.StringIO()
    df.info(buf=buf)
    df_info_str = buf.getvalue()

    # Create the response using the Responses API
    response = openai_client.responses.create(
        model="gpt-4o",
        input=[
            {
                "role": "developer",
                "content": [
                    {"type": "input_text", "text": system_prompt},
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": user_input},
                ],
            },
        ],
    )
    code = response.output_text
    return code

In [ ]:
import plotly.graph_objects as go

# user_input = "Show a bar plot of the top 10 most sold pizzas based on pizza name. Display also the ingredients of those pizzas."
# user_input = "Show pizza ingredients as a pie chart. Only include ingredients that appear in more than 2% of total sales."
# user_input = "Create a time series line chart of total sales per month over the entire dataset."
# user_input = "Plot total revenue per pizza category as a horizontal bar chart, sorted by revenue."
# user_input = "Display a stacked bar chart of pizza quantities sold per category, broken down by pizza size."
# user_input = "Generate a line chart showing the cumulative revenue over time for the top 3 most popular pizza categories."
# user_input = "Show orders per hour throughout the day."
# user_input = "Create a heatmap of average quantity sold by pizza size and pizza category."
user_input = "Compare monthly sales trends of 'The Hawaiian Pizza' and 'The Classic Deluxe Pizza' using a dual-line chart with date on the x-axis."

code = generate_plot(df, user_input)
print(code)
fig = None
exec(code)
fig.show()